In [9]:
##Modified from a ChatGPT draft.
##Prompt: Using the uniprot ID as an input, make an input file for locally-installed 
## Boltz-1 with the protein sequence and post-translational modification data from uniprot, 
## including phosphorylation, methylation, glycosylation, and cross-linking. 


import requests
import os
import re

def download_uniprot_fasta(uniprot_id, fasta_outfile):
    """
    Downloads the FASTA sequence for a given UniProt ID.
    
    Args:
        uniprot_id (str): The UniProt accession number.

    Returns:
        str: The FASTA formatted protein sequence.
    """
    url = f"https://rest.uniprot.org/uniprotkb/{uniprot_id}.fasta"
    response = requests.get(url)
    
    if response.status_code == 200:
        with open(fasta_outfile, "w") as file:
            file.write(response.text)
    else:
        raise ValueError(f"Error downloading sequence for {uniprot_id}: {response.status_code}")

def append_matching_parts(text, possibilities, result_list):
    """
    Appends the parts of a string that match a list of possibilities to a result list.

    Args:
        text: The string to search within.
        possibilities: A list of possible strings to match.
        result_list: The list to append the matching parts to.
    """
    
    pattern = r"(" + "|".join(re.escape(p) for p in possibilities) + r")"
    
    for match in re.finditer(pattern, text):
        result_list.append(match.group(0))



def file_to_list(file_path):
    lines = []
    try:
        with open(file_path, 'r') as file:
            for line in file:
                lines.append(line.rstrip('\n'))
    except FileNotFoundError:
        print(f"Error: File not found: {file_path}")
    return lines

# Dictionary to map common PTMs to their CCD identifiers
ptm_to_ccd = {
    "phosphorylation": {
        "phosphoserine": "phs",     # Phosphorylation on serine
        "phosphothreonine": "pht",  # Phosphorylation on threonine
        "phosphotyrosine": "phty"   # Phosphorylation on tyrosine
    },
    "methylation": {
        "methylation (Lysine)": "me",       # Methylation on lysine
        "methylation (Arginine)": "mearg",  # Methylation on arginine
        "dimethylation (Lysine)": "me2",    # Dimethylation on lysine
        "trimethylation (Lysine)": "me3"    # Trimethylation on lysine
    },
    "glycosylation": {
        "N-linked Glycosylation": "ngly",   # N-linked glycosylation
        "O-linked Glycosylation": "ogly"    # O-linked glycosylation
    },
    "acetylation": {
        "acetylation (Lysine)": "ace"  # Acetylation on lysine
    },
    "ubiquitination": {
        "ubiquitination": "ubiq"  # Ubiquitin attachment
    },
    "sumoylation": {
        "sumoylation": "sumo"  # SUMO attachment
    },
    "crosslinking": {
        "disulfide Bond": "dsb",  # Disulfide bond
        "isopeptide Bond": "iso"  # Isopeptide bond
    }
}

seqGLP1R = { "Tirzepatide" : "YAEGTFTSDYSIALDKIAQKAFVQWLIAGGPSSGAPPPS", \
           "Exenatide" : "HGEGTFTSDLSKQMEEEAVRLFIEWLKNGGPSSGAPPPS", \
           "Liraglutide" : "HAEGTFTSDVSSYLEGQAAKEEFIAWLVRGRG", \
           "Lixisenatide" : "HGEGTFTSDLSKQMEEEAVRLFIEWLKNGGPSSGAPPSKKKKKK", \
           "Albiglutide" : "HGEGTFTSDVSSYLEGQAAKEFIAWLVKGRHGEGTFTSDVSSYLEGQAAKEFIAWLVKGR" \
           "DAHKSEVAHRFKDLGEENFKALVLIAFAQYLQQCPFEDHVKLVNEVTEFAKTCVADESAE" \
           "NCDKSLHTLFGDKLCTVATLRETYGEMADCCAKQEPERNECFLQHKDDNPNLPRLVRPEV" \
           "DVMCTAFHDNEETFLKKYLYEIARRHPYFYAPELLFFAKRYKAAFTECCQAADKAACLLP" \
           "KLDELRDEGKASSAKQRLKCASLQKFGERAFKAWAVARLSQRFPKAEFAEVSKLVTDLTK" \
           "VHTECCHGDLLECADDRADLAKYICENQDSISSKLKECCEKPLLEKSHCIAEVENDEMPA" \
           "DLPSLAADFVESKDVCKNYAEAKDVFLGMFLYEYARRHPDYSVVLLLRLAKTYETTLEKC" \
           "CAAADPHECYAKVFDEFKPLVEEPQNLIKQNCELFEQLGEYKFQNALLVRYTKKVPQVST" \
           "PTLVEVSRNLGKVGSKCCKHPEAKRMPCAEDYLSVVLNQLCVLHEKTPVSDRVTKCCTES" \
           "LVNRRPCFSALEVDETYVPKEFNAETFTFHADICTLSEKERQIKKQTALVELVKHKPKAT" \
           "KEQLKAVMDDFAAFVEKCCKADDKETCFAEEGKKLVAASQAALGL", \
           "Dulaglutide" : "HGEGTFTSDVSSYLEEQAAKEFIAWLVKGGGGGGGSGGGGSGGGGSAESKYGPPCPPCPA" \
           "PEAAGGPSVFLFPPKPKDTLMISRTPEVTCVVVDVSQEDPEVQFNWYVDGVEVHNAKTKP" \
           "REEQFNSTYRVVSVLTVLHQDWLNGKEYKCKVSNKGLPSSIEKTISKAKGQPREPQVYTL" \
           "PPSQEEMTKNQVSLTCLVKGFYPSDIAVEWESNGQPENNYKTTPPVLDSDGSFFLYSRLT" \
           "VDKSRWQEGNVFSCSVMHEALHNHYTQKSLSLSLG", \
           "Semaglutide" : "HAEGTFTSDVSSYLEGQAAKEFIAWLVRGRG", \
            ##Photorhabdus Aegyptia
           "A0A2K2CCW6" : "MTCFSFLCGRRIDSSQESQHMGRDDEELGSIKNVRCYTYRELRNATEGFS" \
           "AANKIGEGGFGSVYKGRLKHGKIAAIKVLSAESRQGVEEFLAEIKAMSEIEHENLVKLYGCCVEDN" \
           "HRILVYNYLENNSLAQTLLDGGHSHSNIQFSWRTRTKICIGVARGLTFLHEEVKPYIVHRDIKASN" \
           "ILLDKDLTAKISDFGLAKLIPDNQTHVSTRVAGTLGYLAPEYAIRGKLTRKADLYSFGVLLLEIVS" \
           "GRNNTNTRLPVEEQYLLERTWELYERRELVSLVDASLNGDFNAEEACRFLKIGLLCTQDDPNLRPSM" \
           "STVVKMLTGRKNFDERKITKPGLISDFMDLKVRAPSKTKASASTSFNVSSGSDNQDTSILTSENSSSV" \
           "TMTAFTELYNRSI", \
           "A0A7X5QNK9" : "MFNLSIQSANTRIDINHNELSQATSTDKTCHCCEPLLNNTIDNPLTAPTSVIT" \
           "QWNEKSVVTNKQIEANISRLASESTAAHKTVDSLLSNLVKLFVRVEGNELTAITKMLNAYTGDKPGFS" \
           "ILGQLGAGGFTRIMSEAQQSGVRPDRANTEPLTLREKATAYYDLTNSAYFRDTLEKIYSSPELKSEFK" \
           "DIIDIGYLESKNFAPARGSTKENPLPDQLALYTFKNENKFNASENSELYQVTKQEKGKEVISNPALAG" \
           "NSLTKVQNDFRTPAPDKDSVWSSAASLEADNRLTNRELEFARLNPRNRLDRTDYQFGQSGPIKAHQEL" \
           "AKENIIVQRGNGFAVWNVKENTGFSKDTALHNLPTVAAPSGTTDRFITAARLLGTGLKNDLSLGTPA" \
           "NGESPEQSIQRGEREMKELTRWLATGYLVDDNHHSMIEVNLGAANHGLESQWGLNLYTEPFSSPIHA" \
           "KGFSISSQEILAELENREDVRTDYSTFKKDLYGGSRAVVNANGSIRTNSR", 
           "A0A022PL40" : "MPDFSIQRANTRIDIHQTELHQPTATDKTSHCCEPLLNNTIDNPLTAPTSVIT" \
           "RWNEKSVVTNKQIETNISRLASESTAAHKTVDSLLSNLVKLFVLAEGNELTAVTKMLNAYTGDKPGFS" \
           "IVGQLGAGGFTRIISEARQNGVRPDRADTEPLTLREKATAYYDLTNSAYFRDTLEKIYSSPELKSEFK" \
           "DIIDIGYLESKNFAPARGSTKEKPLPDQLNLYTFKNENKLNASENPELYQVTKQKDGKEVISNPALAG" \
           "NSLTEVQRDFRTAAPDKDSVWATAASLEADNRLTNRELAFASLNPRNRLDRTDYQFGRSEPIKAHQEL" \
           "AKENIIVQRGNGFSVWNVKENTGFSQDAALHNLPTVAAPSGTTDRFITAARLLGAGLKNDLALGTPTN" \
           "GESTEQSIQRGDREMKELTRWLATGYLVDDNHHSMIEVNLGAANHGLAPQWGLNLYTEPFSSPIYAKG" \
           "FSVSSQEILAELEGRDDVHTDYSTFRKDLYGGNRATVNADGSIKTSSK", 
           "A0A329XCW0" : "MPDFSIQRANTRIDIHQTELHQPTATDKTCHCCEPLLNNTIDNPLTAPTSVITG" \
           "WNEKSVVANKQIEANISRLASESTAAHKTVDSLLSNLVKLFVRAEGNELTAITKMLNAYTGDKPGFSI" \
           "VGQLGAGGFTRIISEARQNGVRPDRANTEPLTLREKATAYYDITNSAYFRDTLEKVYSSPELKSEFKD" \
           "IIDIGYLESKNFAPARGSTKEKPLPDQLALYTFKNENKFNASENPELYQVTKQANGQEVISNPALAGN" \
           "SLTKVQNDFRAPAPDKDSVWSTAASLEADNRLTNRELAFASLNPRNRLDRTDYQFGQSEPIKAHQELA" \
           "KENIIVQRGNGFSVWNVKENTGFSKDAALHNLPTVAAPSGTTDRFITAARLLSAGLKNDLALGAPTNG" \
           "ESPEQSIQRGEREMKELTRWLATGYLVDDNHHSMIEVNLGAANHGLAPQWGLNLYTEPFSSPIHAKGF" \
           "SVSSQEILAELEGRDDVHTDYSIFRKDLYGGNRATVNADGSIKTSSK"
}

#Dictionary to convert the number of entities to their alphabetic equivalents
def to_alphabet_equiv(n):
    result = ''
    while n > 0:
        n, remainder = divmod(n - 1, 26)
        result = chr(65 + remainder) + result
    return result
entity_to_id={i: to_alphabet_equiv(i) for i in range(1,200)}
#entity_to_id={i: chr(64 + i) for i in range(1, 27)}

# Example usage: Retrieve CCD identifier for Phosphoserine
ptm_type = "phosphorylation"
modification = "phosphoserine"
ccd_id = ptm_to_ccd[ptm_type][modification]
#print(f"The CCD identifier for {modification} is: {ccd_id}")



# Function to fetch protein data from UniProt
def fetch_uniprot_data(uniprot_id):
    # Define the UniProt API endpoint for retrieving protein data in JSON format
    url = f"https://www.uniprot.org/uniprot/{uniprot_id}.json"
    response = requests.get(url)
    
    if response.status_code != 200:
        print(f"Error: Unable to fetch data for UniProt ID {uniprot_id}.")
        return None
    
    # Parse the JSON response
    data = response.json()
    
    # Extract protein sequence and PTMs from the response
    sequence = data['sequence']['value']
    features = data.get('features', [])

    #print(features)
    
    ptms = {
        'phosphorylation': [],
        'methylation': [],
        'glycosylation': [],
        'constraints': [],
        'crosslinking': []
    }
    
    # Collect the PTM data
    for feature in features:
        feature_type = feature['type']
        description = feature.get('description', '').lower()
        if feature['type'] == 'Modified residue':
            if 'phosphothreonine' in description or 'phosphoserine' in description or 'phosphotyrosine' in description:
                text = description
                possibilities = ['phosphothreonine', 'phosphoserine', 'phosphotyrosine']
                result_list = []
                append_matching_parts(text, possibilities, result_list)
                ptms['phosphorylation'].append({
                'position': feature['location']['start']['value'],
                'modification': result_list[0]
            })
        elif feature['type'] == 'Methylation':
            ptms['methylation'].append({
                'position': feature['location']['start'],
                'modification': feature['type']
            })
        elif feature['type'] == 'Glycosylation':
            ptms['glycosylation'].append({
                'position': feature['location']['start'],
                'modification': feature['type'],
                'description': feature['description']
            })
            ptms['constraints'].append({
                'position': feature['location']['start'],
                'modification': feature['type']
            })
            
        elif feature['type'] == 'Cross-link':
            ptms['crosslinking'].append({
                'position': feature['location']['start'],
                'modification': feature['type']
            })
    return sequence, ptms

# Function to create the Boltz-1 input file
def create_boltz1_input(entities, glp1r, sequence, ptms, output_filename):
    gly_constraints=[]
    # Create a string for the input file
    input_content = "version: 1\n"
    input_content += "sequences:\n"
    input_content += "  - protein:\n"
    input_content += "      id: " + str(entity_to_id[entities]) + "\n"
    input_content += f"      sequence: {seqGLP1R[glp1r]}\n"
    input_content += "      msa: /lustre/orion/proj-shared/syb111/Personal/twalker/Combined_Pipeline/Sequence_Alignments/" + glp1r + ".a3m\n"
    entities+=1


    
    input_content += "  - protein:\n"
    input_content += "      id: " + str(entity_to_id[entities]) + "\n"
    input_content += f"      sequence: {sequence}\n"
    input_content += "      msa: /lustre/orion/proj-shared/syb111/Personal/twalker/Combined_Pipeline/Sequence_Alignments/" + uniprot_id + ".a3m\n"
    entity_for_mod=entities
    # Add PTM information to the input

    # Phosphorylation
    if ptms['phosphorylation']:
        input_content += "    modifications:\n"
        for mod in ptms['phosphorylation']:
            input_content += f"      - position: {mod['position']}"
            input_content += "\n"
            input_content += "        ccd: "
            input_content += ptm_to_ccd['phosphorylation'][mod['modification']]
            input_content += "\n"
            
    # Methylation
    #if ptms['methylation']:
    #    input_content += "Methylation:\n"
    #    for mod in ptms['methylation']:
    #        input_content += f"  Position: {mod['position']}, Modification: {mod['modification']}\n"
    
    # Glycosylation
    if ptms['glycosylation']:
        for mod in ptms['glycosylation']:
            entities += 1
            input_content += f"  - ligand: \n" 
            #{mod['position']}
            #Note: This assumes that glycosylation occurs between nitrogen and anomeric carbon,
            # nitrogen from the amino acid residue and the glycan's anomeric carbon.
            input_content += "      id: "  + str(entity_to_id[entities]) + "\n"
            if "N-linked (GlcNAc" in mod['description']:
                input_content += "      ccd: "
                input_content += "NAG \n"
                gly_constraints.append([str(entity_to_id[entity_for_mod]), mod['position']['value'], 'N'])
                gly_constraints.append([str(entity_to_id[entities]), 1, 'C1'])
            if "O-linked (GlcNAc" in mod['description']:
                input_content += "      ccd: "
                input_content += "NAG \n"
                gly_constraints.append([str(entity_to_id[entity_for_mod]), mod['position']['value'], 'O'])
                gly_constraints.append([str(entity_to_id[entities]), 1, 'C1'])
    
    # Cross-linking
    #if ptms['crosslinking']:
    #    input_content += "Cross-linking:\n"
    #    for mod in ptms['crosslinking']:
    #        input_content += f"  Position: {mod['position']}, Modification: {mod['modification']}\n"
    
    if ptms['glycosylation']: #or ptms['crosslinking']: not yet implemented
        i=0
        input_content += f"constraints: \n" 
        for mod in ptms['glycosylation']:
            try:
                input_content += f"  - bond: \n" 
                input_content += "      atom1" + ": [" + \
                str(gly_constraints[i]).replace('\'', '').replace('[', '').replace(']', '') + "]\n"
                i += 1
                input_content += "      atom2" + ": [" + \
                str(gly_constraints[i]).replace('\'', '').replace('[', '').replace(']', '') + "]\n"
                i += 1
            except IndexError:
                continue

            
            
    
    # Write to the output file
    print(input_content)
    #print(gly_constraints)
    with open(output_filename, 'w') as f:
        f.write(input_content)
    
    print(f"Boltz-1 input file has been created: {output_filename}")

# Example usage
#   1) Assign the GLP1R agonist to pull its sequence from the seqGLP1R dictionary.
# Options include: 'Tirzepatide', 'Exenatide', 'Liraglutide', 
#'Lixisenatide', 'Albiglutide', 'Dulaglutide', 'Semaglutide.' 
# Use seqGLP1R.keys() to see all options.






In [13]:
uniProtIDs='All_OA_proteins.txt'
glp1r="Tirzepatide"
output_prefix = '/Users/ntw/Desktop/OA_Related_Genes_Tirzepatide/'
suffix= '.yaml'

file_path = output_prefix + uniProtIDs
protList = file_to_list(file_path)
#print(my_list)

for i in range(len(protList)):
    uniprot_id = (protList[i])
    
    output_filename = output_prefix + uniprot_id + "_" + glp1r + "/job" + suffix
    output_directory = output_prefix + uniprot_id + "_" + glp1r + "/"
    fasta_outfile = output_directory +"job.fasta"
    print(output_filename)
    
    os.makedirs(output_directory, exist_ok=True)
    entities=0
    data = fetch_uniprot_data(uniprot_id)
    
    if data:
        sequence, ptms = data
        entities += 1
    
        #print(ptms)
        # Create the Boltz-1 input file
        create_boltz1_input(entities, glp1r, sequence, ptms, output_filename)
        download_uniprot_fasta(uniprot_id, fasta_outfile)

/Users/ntw/Desktop/OA_Related_Genes_Tirzepatide/P49841_Tirzepatide/job.yaml
version: 1
sequences:
  - protein:
      id: A
      sequence: YAEGTFTSDYSIALDKIAQKAFVQWLIAGGPSSGAPPPS
      msa: /lustre/orion/proj-shared/syb111/Personal/twalker/Combined_Pipeline/Sequence_Alignments/Tirzepatide.a3m
  - protein:
      id: B
      sequence: MSGRPRTTSFAESCKPVQQPSAFGSMKVSRDKDGSKVTTVVATPGQGPDRPQEVSYTDTKVIGNGSFGVVYQAKLCDSGELVAIKKVLQDKRFKNRELQIMRKLDHCNIVRLRYFFYSSGEKKDEVYLNLVLDYVPETVYRVARHYSRAKQTLPVIYVKLYMYQLFRSLAYIHSFGICHRDIKPQNLLLDPDTAVLKLCDFGSAKQLVRGEPNVSYICSRYYRAPELIFGATDYTSSIDVWSAGCVLAELLLGQPIFPGDSGVDQLVEIIKVLGTPTREQIREMNPNYTEFKFPQIKAHPWTKVFRPRTPPEAIALCSRLLEYTPTARLTPLEACAHSFFDELRDPNVKLPNGRDTPALFNFTTQELSSNPPLATILIPPHARIQAAASTPTNATAASDANTGDRGQTNNAASASASNST
      msa: /lustre/orion/proj-shared/syb111/Personal/twalker/Combined_Pipeline/Sequence_Alignments/P49841.a3m
    modifications:
      - position: 9
        ccd: phs
      - position: 216
        ccd: phty
      - position: 389
        ccd: phs